### DESAFÍO 07

1. Llamada al data set SVI 2020
2. Llama a la API 

In [2]:
import pandas as pd

url ='C:\\Users\\52444\\Desktop\\Data Science\\DataScienceCoder\\Data Sets\\SVI_2020_US_county.csv'
dfSVI = pd.read_csv(url)
dfSVI.head()

,ST,STATE,ST_ABBR,STCNTY,COUNTY,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,M_TOTPOP,...,EP_ASIAN,MP_ASIAN,EP_AIAN,MP_AIAN,EP_NHPI,MP_NHPI,EP_TWOMORE,MP_TWOMORE,EP_OTHERRACE,MP_OTHERRACE
0,1,Alabama,AL,1001,Autauga,1001,"Autauga County, Alabama",594.455838,55639,0,...,1.2,0.3,0.3,0.2,0.0,0.1,2.0,0.7,0.2,0.3
1,1,Alabama,AL,1003,Baldwin,1003,"Baldwin County, Alabama",1589.835294,218289,0,...,0.9,0.2,0.6,0.2,0.0,0.1,1.9,0.4,0.3,0.2
2,1,Alabama,AL,1005,Barbour,1005,"Barbour County, Alabama",885.007618,25026,0,...,0.5,0.1,0.3,0.3,0.0,0.1,1.3,0.6,0.6,0.4
3,1,Alabama,AL,1007,Bibb,1007,"Bibb County, Alabama",622.469288,22374,0,...,0.3,0.4,0.1,0.1,0.0,0.2,0.4,0.3,0.0,0.2
4,1,Alabama,AL,1009,Blount,1009,"Blount County, Alabama",644.890375,57755,0,...,0.4,0.1,0.1,0.1,0.1,0.1,1.8,0.3,0.3,0.4


In [3]:
import requests
import pandas as pd

# Definir la URL de la API


url = "https://api.census.gov/data/timeseries/healthins/sahie?get=NAME,PCTIC_LB90&for=county:*&time=2020&key=7cef749358cb143c0760b345e66bcf67b4623955"

#https://api.census.gov/data/timeseries/healthins/sahie?get=YEAR&for=county:*&time=2020&key=7cef749358cb143c0760b345e66bcf67b4623955


# Hacer la solicitud GET a la API
response = requests.get(url)

# Verificar si la solicitud fue exitosa (código de estado 200)
if response.status_code == 200:
    # Convertir la respuesta a formato JSON
    data = response.json()
    
    # Crear un DataFrame de pandas con los datos
    df = pd.DataFrame(data[1:], columns=data[0])
    
    # Imprimir la tabla
    display(df)
else:
    # Imprimir un mensaje de error si la solicitud no fue exitosa
    print(f"Error en la solicitud. Código de estado: {response.status_code}")

,NAME,PCTIC_LB90,time,state,county
0,"Autauga County, AL",88.1,2020,01,001
1,"Baldwin County, AL",87.9,2020,01,003
2,"Barbour County, AL",83.7,2020,01,005
3,"Bibb County, AL",85.3,2020,01,007
4,"Blount County, AL",85.2,2020,01,009
...,...,...,...,...,...
3138,"Sweetwater County, WY",84.6,2020,56,037
3139,"Teton County, WY",83.6,2020,56,039
3140,"Uinta County, WY",84.4,2020,56,041
3141,"Washakie County, WY",82.1,2020,56,043


3. Convertir los códigos de estados y condados del df de la API a códigos FIPS. Los códigos FIPS van a ser el Foreign Key para poder unir los dos df.

4. En el df de la API se va a usar la característica de PCTIC_LB90 que es el porcentaje de la población con seguro médico, pero para efectos de este trabajo lo que interesa es el porcentaje de la población que NO tiene seguro médico, se crea una nueva característica llamada API_NOIN con estos datos. 

In [4]:
import pandas as pd

df['state'] = df['state'].astype(str).str.lstrip('0')

df['FIPS'] = df['state'] + df['county']


df['API_NOIN'] = 100 - df['PCTIC_LB90'].astype(float)

# Imprimir el DataFrame actualizado
print(df)


                       NAME PCTIC_LB90  time state county   FIPS  API_NOIN
0        Autauga County, AL       88.1  2020     1    001   1001      11.9
1        Baldwin County, AL       87.9  2020     1    003   1003      12.1
2        Barbour County, AL       83.7  2020     1    005   1005      16.3
3           Bibb County, AL       85.3  2020     1    007   1007      14.7
4         Blount County, AL       85.2  2020     1    009   1009      14.8
...                     ...        ...   ...   ...    ...    ...       ...
3138  Sweetwater County, WY       84.6  2020    56    037  56037      15.4
3139       Teton County, WY       83.6  2020    56    039  56039      16.4
3140       Uinta County, WY       84.4  2020    56    041  56041      15.6
3141    Washakie County, WY       82.1  2020    56    043  56043      17.9
3142      Weston County, WY       85.0  2020    56    045  56045      15.0

[3143 rows x 7 columns]


5. Se hace un MERGE de los dos df creando un nuevo df con las columnas deseadas llamado dfINDEX

In [5]:
import pandas as pd

# Convertir la columna FIPS a tipo string en ambos DataFrames
df['FIPS'] = df['FIPS'].astype(str)
dfSVI['FIPS'] = dfSVI['FIPS'].astype(str)

# Crear el nuevo DataFrame dfINDEX
dfINDEX = pd.merge(df, dfSVI, on='FIPS')

# Seleccionar las columnas deseadas
columns_to_select = ['FIPS', 'API_NOIN', 'EP_POV150', 'EP_UNEMP', 'EP_NOHSDP', 'EP_AGE65', 'EP_AGE17', 'EP_SNGPNT', 'EP_LIMENG', 'EP_MOBILE', 'EP_NOVEH']
dfINDEX = dfINDEX[columns_to_select]

# Imprimir el DataFrame dfINDEX
print(dfINDEX)


       FIPS  API_NOIN  EP_POV150  EP_UNEMP  EP_NOHSDP  EP_AGE65  EP_AGE17  \
0      1001      11.9       22.9       2.9       11.3      15.3      23.6   
1      1003      12.1       16.9       3.9        9.5      20.5      21.5   
2      1005      16.3       40.6       6.9       25.3      19.1      20.9   
3      1007      14.7       27.6       7.4       19.1      16.4      20.5   
4      1009      14.8       23.8       5.2       17.2      18.0      23.2   
...     ...       ...        ...       ...        ...       ...       ...   
3138  56037      15.4       18.0       6.2        7.4      12.2      26.1   
3139  56039      16.4       11.9       1.8        5.3      14.9      17.9   
3140  56041      15.6       18.2       4.5        6.9      14.1      28.8   
3141  56043      17.9       19.2       4.2        8.0      21.6      23.8   
3142  56045      15.0       20.3       3.9        6.7      20.3      19.3   

      EP_SNGPNT  EP_LIMENG  EP_MOBILE  EP_NOVEH  
0           7.5        0.